# Part 1

## These are just preliminary tests for now (10/10/2021 4pm)

In [11]:
import warnings
warnings.filterwarnings('ignore')
# Importing standard modules
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import lr

In [12]:
train = pd.read_csv('../data/diabetes/diabetes_train.csv')
test = pd.read_csv('../data/diabetes/diabetes_test.csv')
valid = pd.read_csv('../data/diabetes/diabetes_val.csv')
all_data = pd.concat([train, test, valid], axis=0)

In [23]:
valid.shape

(100, 9)

In [13]:
# Test run with logistic regression model
trainX=train.drop('Outcome',axis=1)
trainY=train['Outcome']

X=trainX.to_numpy()
Y=trainY.to_numpy()

model = lr.LogisticRegression(verbose=True, )
yh = model.fit(X,Y).predict(X)

100000 Iterations
Norm of gradient: 66.49749138475383

Weights: [  55.77546839    7.10427188   -8.34629678   -1.56957379    2.78822622
    2.42073196   55.22146839   -3.76922089 -323.93717164]



In [38]:
# Function to find convergent solution of gradient descent as a function of learning-rate and maximum iterations
"""
Best one so far:
                    L = 0.0002
                    I = 9e6
"""


def optimize(max_iter, learning_rate, pred_data):
    
    # Input data, features and binary labels column
    Xin = train.drop('Outcome',axis=1).to_numpy()
    Yin = train['Outcome'].to_numpy()
    
    # Prediction data, validation set
    Xp = pred_data.drop('Outcome',axis=1).to_numpy()
    Yp = pred_data['Outcome'].to_numpy().ravel()
    
    # Iterate through the input parameters
    for l in learning_rate:
        
        for m in max_iter:
            
            #print("MAX ITERATIONS: ", m)
            print(f'LEARNING RATE: {l} \n')
            model = lr.LogisticRegression(verbose=True, add_bias=True, learning_rate=l, max_iters=m)
            yh = model.fit(Xin,Yin).predict(Xp)
            T = []
            T.append(yh)
        
            # Decision Boundary
            prediction = []
            for x in np.array(T).ravel():
                if x < 0.5: prediction.append(0)
                else: prediction.append(1)

            #print(T)
            #print(prediction)
            #print(Yp)
            print("Accuracy Score:", accuracy_score(Yp,np.array(prediction).ravel()))
    
mi = [1e4,9e4,1e5,9e5,1e6,9e6]
learn = [0.0001,0.0009,0.0002,0.00001,0.001,0.0005]

optimize(max_iter=mi, learning_rate=learn, pred_data=test)

LEARNING RATE: 0.0001 

10000 Iterations
Norm of gradient: 0.05549797321390251

Weights: [ 0.10634008  0.01324276 -0.03067612 -0.00097639  0.00155382 -0.00562319
  0.00618771 -0.00815136 -0.03614793]

Accuracy Score: 0.7205882352941176
90000 Iterations
Norm of gradient: 0.03449646341252793

Weights: [ 0.13165091  0.01419986 -0.02988331 -0.00113209  0.00151228 -0.00272609
  0.05968231 -0.01166192 -0.31324052]

Accuracy Score: 0.7205882352941176
100000 Iterations
Norm of gradient: 0.034307864937074266

Weights: [ 0.13152847  0.014274   -0.02976018 -0.00115303  0.00150239 -0.00238826
  0.06590202 -0.01156154 -0.34707329]

Accuracy Score: 0.7205882352941176
900000 Iterations
Norm of gradient: 0.02240316810578086

Weights: [ 1.27120605e-01  1.97367024e-02 -2.29781230e-02 -2.08817141e-03
  8.95230229e-04  2.03927523e-02  3.56405128e-01 -4.82386649e-03
 -2.55614319e+00]

Accuracy Score: 0.7352941176470589
1000000 Iterations
Norm of gradient: 0.021300042953086067

Weights: [ 1.27088357e-01  2.

9000000 Iterations
Norm of gradient: 34.98848504133575

Weights: [ 2.09305029e-01  5.70107269e-02 -1.34325514e-02 -6.97254908e-03
  6.91086783e-03  1.24627387e-01  1.20803925e+00  1.85009852e-02
 -1.19250813e+01]

Accuracy Score: 0.7352941176470589
